# Watchlist example notebook

Here we will do a xmatch against a list of positions in the sky, given in the file ```watchlist.csv```

In [1]:
import pandas as pd
import psycopg2

# Get credentials to open a direct connection with the database

In [2]:
import json
# use the location of your credentials file here
credentials_file = "/home/fforster/Dropbox/Work/alercebroker/usecases/alercereaduser_v4.json"
with open(credentials_file) as jsonfile:
    params = json.load(jsonfile)["params"]

In [3]:
conn = psycopg2.connect(dbname=params["dbname"], user=params["user"], host=params["host"], password=params["password"])

# Get watchlist of objects

In [4]:
df = pd.read_csv("data/watchlist.csv")
df.head()

,id_source,ra,dec
0,source_1,160.183014,33.016467
1,source_2,174.215249,44.837895


# Prepare the query string for watchlist for object within 1 degress of given positions, and  happening within the last 10 days

In [5]:
objects = []
for _,row in df.iterrows():
    objects.append(f"(\'{row.id_source}\', {row.ra}, {row.dec})")
objects_str = ",\n".join(objects)
objects_str

"('source_1', 160.18301441363647, 33.0164673528409),\n('source_2', 174.21524897555543, 44.83789535222221)"

Create query string

In [6]:
from astropy.time import Time
nt = Time.now()
last_mjd_discovery = nt.mjd - 10

In [7]:
search_radius = 1 # degrees
query = """
WITH catalog ( source_id, ra, dec) AS (
    VALUES
        {values}
)
SELECT 
    c.source_id, c.ra, c.dec, o.oid, o.meanra, o.meandec, q3c_dist(c.ra,c.dec,o.meanra,o.meandec), 
    o.firstmjd

FROM object o, catalog c
    /*
     * It is REALLY important to first use the catalog then the object ra,dec for speed. The radius is in degrees.
     */
WHERE
    q3c_join(c.ra, c.dec,o.meanra, o.meandec, {radius})
    AND o.firstmjd > %s
""" % (last_mjd_discovery)
query_str = query.format(values = objects_str, radius = search_radius) # radius in degrees

In [8]:
matches = pd.read_sql(query_str,conn)
print(matches.shape)
matches

(53, 8)


,source_id,ra,dec,oid,meanra,meandec,q3c_dist,firstmjd
0,source_2,174.215249,44.837895,ZTF21aaqpmcu,174.034389,45.460728,0.635760,59297.266782
1,source_2,174.215249,44.837895,ZTF21aaqpmek,174.420623,44.382857,0.477949,59297.266782
2,source_1,160.183014,33.016467,ZTF21aaqpmiy,160.841143,32.876399,0.569772,59297.268715
3,source_1,160.183014,33.016467,ZTF21aaqwdiv,159.371305,32.521336,0.843207,59301.299630
4,source_1,160.183014,33.016467,ZTF21aaqwdiu,159.756359,32.659909,0.505608,59301.299630
5,source_1,160.183014,33.016467,ZTF21aaqwdki,160.806774,32.632821,0.649563,59301.299630
6,source_1,160.183014,33.016467,ZTF21aaqypmo,159.991994,32.595622,0.450431,59303.259306
7,source_1,160.183014,33.016467,ZTF21aaqypmp,160.168464,32.870279,0.146698,59303.259306
8,source_1,160.183014,33.016467,ZTF21aaqypum,160.632521,33.691913,0.772786,59303.259306
9,source_1,160.183014,33.016467,ZTF21aaqzdqj,160.607393,32.423670,0.692014,59303.323958


# Query objects within 10 degress of given positions, first detected within the last 10 days, with SN probabilities > 0.4, and ranking=1 in the stamp classifier

In [9]:
search_radius = 10 # degrees
query = """
WITH catalog ( source_id, ra, dec) AS (
    VALUES
        {values}
),
sn (oid, classifier_name, class_name, probability, ranking) AS (
    SELECT
        o.oid, p.classifier_name, p.class_name, p.probability, p.ranking
    FROM
        probability p
    INNER JOIN 
        object o
    ON 
        o.oid=p.oid
    WHERE
        p.classifier_name='stamp_classifier'
        AND p.class_name IN ('SN')
        AND p.ranking=1
        AND p.probability > 0.4
        AND o.firstmjd > %s
)

SELECT 
    c.source_id, c.ra, c.dec, o.oid, o.meanra, o.meandec, q3c_dist(c.ra,c.dec,o.meanra,o.meandec), 
    o.firstmjd, sn.classifier_name, sn.class_name, sn.probability, sn.ranking

FROM object o INNER JOIN sn ON sn.oid=o.oid, catalog c
    /*
     * It is REALLY important to first use the catalog then the object ra,dec for speed. The radius is in degrees.
     */
WHERE
    q3c_join(c.ra, c.dec,o.meanra, o.meandec, {radius})
    AND o.firstmjd > %s
ORDER BY
    c.source_id
""" % (last_mjd_discovery, last_mjd_discovery)
query_str = query.format(values = objects_str, radius = search_radius) # radius in degrees

In [10]:
matches = pd.read_sql(query_str,conn)
print(matches.shape)
matches

(15, 12)


,source_id,ra,dec,oid,meanra,meandec,q3c_dist,firstmjd,classifier_name,class_name,probability,ranking
0,source_1,160.183014,33.016467,ZTF21aaqyqwk,157.083206,39.806022,7.231987,59303.273889,stamp_classifier,SN,0.755601,1
1,source_1,160.183014,33.016467,ZTF21aarngcx,163.635601,38.153172,5.852816,59304.217732,stamp_classifier,SN,0.685881,1
2,source_1,160.183014,33.016467,ZTF21aaroydq,163.573443,34.238747,3.076068,59306.304271,stamp_classifier,SN,0.524896,1
3,source_1,160.183014,33.016467,ZTF21aaqyraq,156.888685,42.095899,9.445432,59303.273889,stamp_classifier,SN,0.571711,1
4,source_1,160.183014,33.016467,ZTF21aardzhu,163.387745,40.903619,8.290720,59304.174120,stamp_classifier,SN,0.404721,1
5,source_1,160.183014,33.016467,ZTF21aaqzdut,157.638588,39.106205,6.426905,59303.273889,stamp_classifier,SN,0.818674,1
6,source_1,160.183014,33.016467,ZTF21aardzlu,155.949606,42.275886,9.843913,59306.248542,stamp_classifier,SN,0.651708,1
7,source_2,174.215249,44.837895,ZTF21aargpjl,185.399322,50.550454,9.433266,59305.282905,stamp_classifier,SN,0.503507,1
8,source_2,174.215249,44.837895,ZTF21aaqzdjg,170.966689,41.935656,3.740514,59297.242882,stamp_classifier,SN,0.472143,1
9,source_2,174.215249,44.837895,ZTF18aawqich,186.975899,43.222875,9.305425,59302.262234,stamp_classifier,SN,0.784540,1
